In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from eolearn.core import (
    AddFeatureTask,
    EONode,
    EOPatch,
    EOTask,
    EOWorkflow,
    FeatureType,
    LoadTask,
    OverwritePermission,
    SaveTask,
)

In [3]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

In [4]:
from pathlib import Path
import sys

SCRIPT_DIR = Path('/home/alina.smolina/eolearn-pipeline/src')
sys.path.append(str(SCRIPT_DIR))
SCRIPT_DIR = Path('/home/alina.smolina/DL-pipeline/src')
sys.path.append(str(SCRIPT_DIR))

In [30]:
eopatches_dir = '/beegfs/home/alina.smolina/data/sakhalin/images/EOPatches/train_2560_K_N_v2/'
num_ids_train = 598


eopatches_dir_test = '/beegfs/home/alina.smolina/data/sakhalin/images/EOPatches/test_2560_Nevelsk_v2/'
num_ids_test = 44


# eopatches_dir_test = '/beegfs/home/alina.smolina/data/sakhalin/images/EOPatches/test_2560_Korsak_v2/'
# num_ids_test = 77

## Preparing dataloader

In [9]:
eopatch = EOPatch.load(f'{eopatches_dir}eopatch_20', lazy_loading=True)

In [10]:
eopatch

EOPatch(
  bbox=BBox(((637440.0, 5178880.0), (640000.0, 5181440.0)), crs=CRS('32654'))
  timestamps=[2018-03-14 01:26:50, ...]<length=8>
  data={
    10BANDS: FeatureIONumpy(/data/10BANDS.npy.gz)
    BANDS: FeatureIONumpy(/data/BANDS.npy.gz)
    CLP: FeatureIONumpy(/data/CLP.npy.gz)
  }
  mask_timeless={
    2_GROUPS: FeatureIONumpy(/mask_timeless/2_GROUPS.npy.gz)
  }
  mask={
    CLM: FeatureIONumpy(/mask/CLM.npy.gz)
    IS_DATA: FeatureIONumpy(/mask/IS_DATA.npy.gz)
    SCL: FeatureIONumpy(/mask/SCL.npy.gz)
  }
  vector_timeless={
    5-2groups-32654: FeatureIOGeoDf(/vector_timeless/5-2groups-32654.gpkg.gz)
  }
  scalar={
    COVERAGE: FeatureIONumpy(/scalar/COVERAGE.npy.gz)
  }
)

## Model

In [7]:
from datamodule_eolearn import GeoEOModule

import torch
torch.set_float32_matmul_precision('high')

In [24]:
import albumentations as A
import albumentations.pytorch as AP

transforms = A.Compose([
    A.Flip(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=(-0.0625, 0.0625), 
        scale_limit=0, #no scale
        rotate_limit=(-90, 90), 
        p=0.5
    ),
    AP.ToTensorV2(transpose_mask=True),
    ],
)

test_transform = A.Compose([
    AP.ToTensorV2(transpose_mask=True),
    ],
)

In [31]:
all_dates = GeoEOModule(
    transform = transforms,
    test_transform = test_transform,
    target_mask_name = '2groups',
    train_eopatches_dir = eopatches_dir, 
    test_eopatches_dir = eopatches_dir_test, 
    predict_eopatches_dir = eopatches_dir_test, 
    train_eopatches_ids = [x for x in range(num_ids_train)],
    test_eopatches_ids = [x for x in range(num_ids_test)],
    predict_eopatches_ids = [x for x in range(num_ids_test)], 
    train_date_range = ['2018-01-01', '2018-12-31'],
    test_date_range = ['2018-01-01', '2018-12-31'],
    predict_date_range = ['2018-01-01', '2018-12-31'],
    batch_size = 128,
    num_workers = 24,
)

In [11]:
import wandb
import pytorch_lightning as pl
from module import SegModule

In [27]:
model_inference = SegModule.load_from_checkpoint(
    "/beegfs/home/alina.smolina/DL-pipeline/weights/group-2classes-sakhalin/lucky-mountain-3/epoch=169-val_loss=0.0000.ckpt"
    # "/beegfs/home/alina.smolina/DL-pipeline/weights/group-2groups-sakhalin/glamorous-spaceship-8/epoch=499-val_loss=0.00000.ckpt"
)

In [28]:
from pytorch_lightning.loggers import WandbLogger
import os
os.environ["WANDB__SERVICE_WAIT"] = "500"

wandb_logger = WandbLogger(project='dl-pipeline-sakhalin', log_model=True)
print(f'🤖: Look at {wandb.run.url}')

trainer = pl.Trainer(
    logger=wandb_logger,
)

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:398: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/alina.smolina/.conda/envs/geotorch/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


🤖: Look at https://wandb.ai/nali/dl-pipeline-sakhalin-2groups/runs/u2mv29fz


In [32]:
model_inference.eval()
trainer.test(model_inference, all_dates)

/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/alina.smolina/.conda/envs/geotorch/lib ...
  rank_zero_warn(


🤖: Setup data...
🤖: Collecting all the time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Elapsed time: 0.55 min
Done.
🤖: Loading all the patches and time frames into the memory... 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Done.
Elapsed time: 4.42 min 🤖: Test stage.
🤖: Test frames >>> 787
🤖: #iterations in test dataloader: 7


/trinity/home/alina.smolina/.conda/envs/geotorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 test/f1score_лиственные    0.6971182227134705
  test/f1score_хвойные      0.7202945351600647
      test/mean_acc         0.7086091041564941
test/precision_лиственные   0.7056609392166138
 test/precision_хвойные     0.7123309373855591
 test/recall_лиственные     0.6887799501419067
   test/recall_хвойные      0.7284381985664368
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/mean_acc': 0.7086091041564941,
  'test/precision_лиственные': 0.7056609392166138,
  'test/precision_хвойные': 0.7123309373855591,
  'test/recall_лиственные': 0.6887799501419067,
  'test/recall_хвойные': 0.7284381985664368,
  'test/f1score_лиственные': 0.6971182227134705,
  'test/f1score_хвойные': 0.7202945351600647}]

In [55]:
%%time
uniq_vals = []
# # for item in eo_ds:
all_dates.setup(stage='fit')
for item in all_dates.train_dataloader():
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    uniq_vals.append(torch.unique(item[-1]).numpy())
    break

🤖: Setup data...
🤖: Fit stage.
🤖: Collecting all time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Done.
🤖: Fit stage.
🤖: Fit frames >>> 1485
🤖: Val frames >>> 371
🤖: Total >>>>>>>> 1856
🤖: #iterations in train dataloader: 62
3
torch.Size([24, 10, 256, 256])
torch.Size([24, 256, 256])
CPU times: user 22.7 s, sys: 1.93 s, total: 24.7 s
Wall time: 4min 34s


[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15


In [53]:
%%time
uniq_vals = []
# # for item in eo_ds:
all_dates.setup(stage='fit')
for item in all_dates.train_dataloader():
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    uniq_vals.append(torch.unique(item[-1]).numpy())
    break

🤖: Setup data...
🤖: Fit stage.
🤖: Collecting all time indices... Done.
🤖: Generating mapping: idx -> (patch_id, time_id)... Done.
🤖: Fit stage.
🤖: Fit frames >>> 1485
🤖: Val frames >>> 371
🤖: Total >>>>>>>> 1856
🤖: #iterations in train dataloader: 62
3
torch.Size([24, 10, 256, 256])
torch.Size([24, 256, 256])
CPU times: user 22.7 s, sys: 1.03 s, total: 23.8 s
Wall time: 37.3 s


In [ ]:
uniq_vals

In [20]:
from datamodule_eolearn import EOLearnDataset

eo_ds = EOLearnDataset(
    transform = transforms,
    eopatches_dir = eopatches_dir,
    eopatches_ids = [x for x in range(69, 106)],
    date_range = ['2018-01-01', '2018-12-31'],
    target_mask_name = 'PREVAIL_GROUP_2',
    demand_target = True,
)

🤖: Collecting all time indices...
🤖: Done.


In [25]:
for item in eo_ds:
    print(len(item))
    print(item[0].shape)
    print(item[1].shape)
    break

3
torch.Size([10, 256, 256])
torch.Size([256, 256])
